# MATH541 Data Analysis and Statistical Learning Project 2

## Author: Olzhas Shortanbaiuly

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

My dataset was "Ecoli data set" involving the protein localization sites. It can be accessed in http://archive.ics.uci.edu/ml/datasets/Ecoli

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/oshortanbaiuly/MATH542_proj2/main/ecoli.csv')
data.set_index('SEQUENCE_NAME', inplace=True)
for i in range(len(data.SITE)):
    if data['SITE'][i] != 'cp':
        data['SITE'][i] = 'ncp' #'ncp' stands for 'not cp'

C:\Users\olzhas.shortanbaiuly\AppData\Local\Temp\ipykernel_3880\1165469454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['SITE'][i] = 'ncp' #'ncp' stands for 'not cp'


In [ ]:
data.head()

,MCG,GVH,LIP,CHG,AAC,ALM1,ALM2,SITE
SEQUENCE_NAME,,,,,,,,
AAT_ECOLI,0.49,0.29,0.48,0.5,0.56,0.24,0.35,cp
ACEA_ECOLI,0.07,0.40,0.48,0.5,0.54,0.35,0.44,cp
ACEK_ECOLI,0.56,0.40,0.48,0.5,0.49,0.37,0.46,cp
ACKA_ECOLI,0.59,0.49,0.48,0.5,0.52,0.45,0.36,cp
ADI_ECOLI,0.23,0.32,0.48,0.5,0.55,0.25,0.35,cp


### 1. If you have a large database (N >> 20p), then divide your data into 3 parts: the training set, the validation, and the test set. Decide which ratio train : validation : test to use and explain your motives.

In [ ]:
data.shape

(336, 8)

Since there are 336 points in the dataset and it is a considerably small amount, we can allocate 50% of the data to train the model. Other than that, 25% of data is used to test the model and 25% of data is taken as a validation set.

In [ ]:
# Separate features and target variable
X = data.drop('SITE', axis=1)
y = data['SITE']

# Split data into training, test and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size = 2/3, random_state=1)

## 2. Standardize all you features X1, ..., Xp. Standardization of feature Xi meansreplacing its values with $\frac{X_i - \bar{X_i}}{\sqrt{\bar{X_{i}^2}-(\bar{X_i})^2}}$ where $\bar{f}$ denotes averaging over your training sample.

In [ ]:
# Transform the training, validation, and test sets using the fitted scaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train_std = scaler.transform(X_train)
X_valid_std = scaler.transform(X_valid)
X_test_std = scaler.transform(X_test)

## 3. Choose 3-5 candidates for C : C1, C2, ..., C5 and find weight vectors (using only training set) for Model I (SVM model with linear kernel)

In [ ]:
# Candidate C values
C_values = [0.1, 1, 10, 100, 1000]

# Create an empty list to store the models
models = []

# Train a model for each candidate C value
for C in C_values:
    # Create an SVM model with a linear kernel and the current C value
    model = SVC(kernel='linear', C=C, random_state=1)
    model.fit(X_train_std, y_train)
    models.append(model)

# Print weight vectors for each model
for i, model in enumerate(models):
    print(f"Weight vector for C = {C_values[i]}:\n{model.coef_}\n")

Weight vector for C = 0.1:
[[0.45459437 0.69906646 0.00482766 0.01066491 0.27703884 1.10063302
  0.01969326]]

Weight vector for C = 1:
[[ 6.12386491e-01  8.17731699e-01  5.55111512e-17  0.00000000e+00
   2.91512302e-01  2.33019328e+00 -7.94256669e-01]]

Weight vector for C = 10:
[[ 7.06160956e-01  8.41713142e-01  4.44089210e-16  1.77635684e-15
   1.53280955e-01  3.48325286e+00 -1.58982448e+00]]

Weight vector for C = 100:
[[ 6.78633426e-01  8.21703797e-01 -2.84217094e-14 -2.66453526e-14
   1.71165118e-01  3.47537222e+00 -1.67458070e+00]]

Weight vector for C = 1000:
[[ 5.70305218e-01  9.32893535e-01 -9.94759830e-13 -3.69482223e-13
   3.11661110e-01  4.32394479e+00 -2.33904883e+00]]



## 4. For C = $C_i$, estimate the following values on the training set: (a) percent of incorrectly classified examples (error rate) (b) precision and recall (c) the number of support vectors.


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Iterate over the trained models and calculate the metrics
for i, model in enumerate(models):
    # Predict the classes using the current model
    y_pred = model.predict(X_train_std)

    # Calculate the error rate
    error_rate = 1 - accuracy_score(y_train, y_pred)

    # Calculate the precision and recall
    precision = precision_score(y_train, y_pred, pos_label='ncp')
    recall = recall_score(y_train, y_pred, pos_label='ncp')

    # Get the number of support vectors
    n_support_vectors = len(model.support_)

    # Print the results
    print(f"For C = {C_values[i]}:")
    print(f"  (a) Error rate: {error_rate:.2f}")
    print(f"  (b) Precision: {precision:.2f}, Recall: {recall:.2f}")
    print(f"  (c) Number of support vectors: {n_support_vectors}\n")

For C = 0.1:
  (a) Error rate: 0.05
  (b) Precision: 0.97, Recall: 0.95
  (c) Number of support vectors: 49

For C = 1:
  (a) Error rate: 0.04
  (b) Precision: 0.98, Recall: 0.96
  (c) Number of support vectors: 28

For C = 10:
  (a) Error rate: 0.04
  (b) Precision: 0.96, Recall: 0.97
  (c) Number of support vectors: 20

For C = 100:
  (a) Error rate: 0.04
  (b) Precision: 0.97, Recall: 0.97
  (c) Number of support vectors: 20

For C = 1000:
  (a) Error rate: 0.03
  (b) Precision: 0.98, Recall: 0.97
  (c) Number of support vectors: 18



## 5. For C = Ci, estimate the following values on the validation set: (a) percent of incorrectly classified examples (error rate) (b) precision and recall (c) Choose proper β (natural to your problem) and estimate $F_β$ measure: $F_{\beta} = (1 + \beta^2) \cdot \frac{precision \cdot recall}{\beta^2 \cdot precision + recall}$. Collect all your findings into a table. Using those estimations, choose the best value $C = C_{i}^∗$

In [ ]:
# Defining the chosen beta value
beta = 0.5  # Other values can be chosen depending on the emphasis you want to give to precision or recall

# Create a placeholder for the best F_beta score and best C value
best_f_beta = -1
best_c = None

# Iterate over the trained models and calculate the metrics on the validation set
for i, model in enumerate(models):
    # Predict the classes using the current model
    y_pred_valid = model.predict(X_valid_std)

    # Calculate the error rate
    error_rate_valid = 1 - accuracy_score(y_valid, y_pred_valid)

    # Calculate the precision and recall
    precision_valid = precision_score(y_valid, y_pred_valid, pos_label='ncp')
    recall_valid = recall_score(y_valid, y_pred_valid, pos_label='ncp')

    # Calculate the F_beta measure
    f_beta = (1 + beta**2) * (precision_valid * recall_valid) / ((beta**2 * precision_valid) + recall_valid)

    # Update the best F_beta score and best C value
    if f_beta > best_f_beta:
        best_f_beta = f_beta
        best_c = C_values[i]

    print(f"For C = {C_values[i]}:")
    print(f"  (a) Error rate on validation set: {error_rate_valid:.2f}")
    print(f"  (b) Precision: {precision_valid:.2f}, Recall: {recall_valid:.2f}")
    print(f"  (c) F_{beta} measure: {f_beta:.2f}\n")

# Print the best C value
print(f"Best C value: {best_c}")

For C = 0.1:
  (a) Error rate on validation set: 0.02
  (b) Precision: 0.96, Recall: 1.00
  (c) F_0.5 measure: 0.97

For C = 1:
  (a) Error rate on validation set: 0.02
  (b) Precision: 0.96, Recall: 1.00
  (c) F_0.5 measure: 0.97

For C = 10:
  (a) Error rate on validation set: 0.02
  (b) Precision: 0.96, Recall: 1.00
  (c) F_0.5 measure: 0.97

For C = 100:
  (a) Error rate on validation set: 0.02
  (b) Precision: 0.96, Recall: 1.00
  (c) F_0.5 measure: 0.97

For C = 1000:
  (a) Error rate on validation set: 0.04
  (b) Precision: 0.94, Recall: 1.00
  (c) F_0.5 measure: 0.96

Best C value: 0.1


As $\beta$ in $F_{\beta}$, 0.5 was chosen due to the specifics of the task. As we aim to localize the protein, performing a classification task on finding the localization site. As the cost of misclassifying a protein to the wrong location is high due to medical risks, we priotize precision over recall. In medical diagnosis tasks as protein localization, it is prioritized to avoid false positives over identifying all possible true positives because false positive diagnosis can lead to unnecessary treatment or surgery, so it may be more important to obtain a high precision in order to avoid false positives.

$\beta = 0.5$ prioritizes precision so this is why it was chosen.

## 6. Choose 3-5 candidates for C : $C_{1}^{'}, C_{2}^{'}, C_{3}^{'}, C_{4}^{'}, C_{5}^{'}$ and find weight vectors (using only training set) for Models II-IV (with polynomial, radial basis function, sigmoid kernels)

In [ ]:
# Candidate C values
C_values_prime = [0.1, 1, 10, 100, 1000]

# Kernels
kernels = ['poly', 'rbf', 'sigmoid']

# Create a dictionary to store the models for each kernel
models_by_kernel = {}

# Train a model for each candidate C value and kernel
for kernel in kernels:
    models = []
    for C in C_values_prime:
        # Create an SVM model with the current kernel and C value
        model = SVC(kernel=kernel, C=C, random_state=1) # Using the most standard case for each kernel

        # Train the model using the standardized training data
        model.fit(X_train_std, y_train)

        # Add the trained model to the list of models
        models.append(model)

    # Store the models for the current kernel
    models_by_kernel[kernel] = models

In [ ]:
# Print weight vectors for the polynomial kernel models
for i, model in enumerate(models_by_kernel['poly']):
    print(f"Weight vector for polynomial kernel with C = {C_values_prime[i]}:\n{model.coef_}\n")

AttributeError: coef_ is only available when using a linear kernel

In [ ]:
# Print weight vectors for the polynomial kernel models
for i, model in enumerate(models_by_kernel['rbf']):
    print(f"Weight vector for polynomial kernel with C = {C_values_prime[i]}:\n{model.coef_}\n")

AttributeError: coef_ is only available when using a linear kernel

In [ ]:
# Print weight vectors for the polynomial kernel models
for i, model in enumerate(models_by_kernel['sigmoid']):
    print(f"Weight vector for polynomial kernel with C = {C_values_prime[i]}:\n{model.coef_}\n")

AttributeError: coef_ is only available when using a linear kernel

As we can see, computing weights for kernels other than 'linear' is not an easy task due to the fact that attribute ".coef_" exists only for linear kernels.

For other kernels, we approximate the weights using the dual coefficients and support vectors. As the weight vector can be thought of as a combination of the support vectors and their corresponding coefficients coming from solving the dual optimization problem.

Even though we know that for an RBF kernel, weights can never be computed analytically due to the fact that it is infinite-dimensional, it can be attempted to give approximate weights.

In [ ]:
w_poly = []
for model in models_by_kernel['poly']:
    support_vectors = model.support_vectors_
    coefficients = model.dual_coef_
    w_poly.append(np.dot(coefficients, support_vectors))

for i, model in enumerate(models_by_kernel['poly']):
    print(f"Weight vector for polynomial kernel with C = {C_values_prime[i]}:\n{w_poly[i]}\n")

Weight vector for polynomial kernel with C = 0.1:
[[6.11673090e+00 4.90609401e+00 3.45413690e-02 2.99651530e-04
  3.49126375e+00 8.36120777e+00 4.37239392e+00]]

Weight vector for polynomial kernel with C = 1:
[[2.53351388e+01 2.15672285e+01 4.63619950e-02 1.42272538e-03
  1.37026709e+01 3.57876593e+01 1.20163449e+01]]

Weight vector for polynomial kernel with C = 10:
[[1.06571989e+02 5.75137455e+01 9.55843636e-03 4.67633027e-03
  5.45822892e+01 1.04865036e+02 3.24252515e+01]]

Weight vector for polynomial kernel with C = 100:
[[5.17823898e+02 1.53884198e+02 3.72582850e-03 8.23082616e-03
  7.03957531e+01 3.07570496e+02 4.22313017e+01]]

Weight vector for polynomial kernel with C = 1000:
[[2.07134942e+03 8.68682543e+02 1.90303939e-02 4.20405459e-02
  4.38304519e+02 1.00883749e+03 7.02402472e+02]]



In [ ]:
w_rbf = []
for model in models_by_kernel['rbf']:
    support_vectors = model.support_vectors_
    coefficients = model.dual_coef_
    w_rbf.append(np.dot(coefficients, support_vectors))

for i, model in enumerate(models_by_kernel['rbf']):
    print(f"Weight vector for rbf kernel with C = {C_values_prime[i]}:\n{w_rbf[i]}\n")

Weight vector for rbf kernel with C = 0.1:
[[3.4010435  3.57652696 2.94239167 1.30002303 1.71476566 4.90132057
  1.30712411]]

Weight vector for rbf kernel with C = 1:
[[3.75763273 4.91617027 2.27258179 1.89706467 0.66197202 8.68843232
  0.80854985]]

Weight vector for rbf kernel with C = 10:
[[ 6.41023967  8.19403648  2.29455689  1.94811612  1.63240696 12.42305856
  -3.05466299]]

Weight vector for rbf kernel with C = 100:
[[ 1.76659005e+01  2.91766842e+01  2.13162821e-14 -1.68753900e-14
   9.89209203e-01  2.02024926e+01 -3.38419437e+00]]

Weight vector for rbf kernel with C = 1000:
[[ 3.27744261e+01  7.83450356e+01  0.00000000e+00 -1.42108547e-14
   5.77017677e+00  4.52082041e+01 -1.43484395e+01]]



In [ ]:
w_sigmoid = []
for model in models_by_kernel['sigmoid']:
    support_vectors = model.support_vectors_
    coefficients = model.dual_coef_
    w_sigmoid.append(np.dot(coefficients, support_vectors))

for i, model in enumerate(models_by_kernel['sigmoid']):
    print(f"Weight vector for sigmoid kernel with C = {C_values_prime[i]}:\n{w_sigmoid[i]}\n")

Weight vector for sigmoid kernel with C = 0.1:
[[2.543242   3.05363792 1.17695667 1.30002303 1.29240835 4.00956398
  0.60788682]]

Weight vector for sigmoid kernel with C = 1:
[[ 3.67533304e+00  5.82407356e+00  8.32667268e-17 -1.52655666e-16
   1.70548964e+00  8.66355873e+00 -1.05973251e-01]]

Weight vector for sigmoid kernel with C = 10:
[[ 1.99457606e+00  6.76288588e+00 -8.88178420e-16 -4.44089210e-16
  -1.65014246e+00  2.57473499e+01 -9.44865460e+00]]

Weight vector for sigmoid kernel with C = 100:
[[ 9.81753286e+01  2.51912099e+02  0.00000000e+00  5.32907052e-15
   9.68121962e+01  2.77951405e+02 -1.63458706e+02]]

Weight vector for sigmoid kernel with C = 1000:
[[ 1.34483350e+03  3.36214311e+03 -1.13686838e-13 -1.42108547e-13
  -1.25816099e+03  3.15542865e+03 -1.05449099e+03]]



## 5. For a Model $\alpha$, where $\alpha$ = II-IV do the following: (a) For $C = C_{i}^{'}$, find a solution $\Lambda_{\alpha,C_{i}^{'}}^{*}$ for Wolfe dual problem (using only training set). (b) Make estimations as at steps 2-3. Using this estimations, choose the best values $C = C_{\alpha}^{'}$ for each of the models.

In [ ]:
beta = 0.5

# Iterate over the kernels and trained models
for kernel, models in models_by_kernel.items():
    print(f"Kernel: {kernel}")

    best_f_beta = -1
    best_c = None

    for i, model in enumerate(models):
        # Access the dual coefficients
        dual_coefs = model.dual_coef_

        # Calculate metrics for the training set
        y_pred_train = model.predict(X_train_std)
        error_rate_train = 1 - accuracy_score(y_train, y_pred_train)
        precision_train = precision_score(y_train, y_pred_train, pos_label='ncp')
        recall_train = recall_score(y_train, y_pred_train, pos_label='ncp')

        # Calculate metrics for the validation set
        y_pred_valid = model.predict(X_valid_std)
        error_rate_valid = 1 - accuracy_score(y_valid, y_pred_valid)
        precision_valid = precision_score(y_valid, y_pred_valid, pos_label='ncp')
        recall_valid = recall_score(y_valid, y_pred_valid, pos_label='ncp')
        f_beta_valid = (1 + beta**2) * (precision_valid * recall_valid) / ((beta**2 * precision_valid) + recall_valid)

        # Update the best F_beta score and best C value if needed
        if f_beta_valid > best_f_beta:
            best_f_beta = f_beta_valid
            best_c = C_values_prime[i]

        # Print the results
        print(f"  For C = {C_values_prime[i]}:")
        print(f"    (a) Dual coefficients: {dual_coefs}")
        print(f"    (b) Training set: Error rate: {error_rate_train:.2f}, Precision: {precision_train:.2f}, Recall: {recall_train:.2f}")
        print(f"        Validation set: Error rate: {error_rate_valid:.2f}, Precision: {precision_valid:.2f}, Recall: {recall_valid:.2f}, F_{beta} measure: {f_beta_valid:.2f}\n")

    # Print the best C value for the current kernel
    print(f"  Best C value for {kernel} kernel: {best_c}\n")

Kernel: poly
  For C = 0.1:
    (a) Dual coefficients: [[-1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -3.85873334e-02 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.00000000e-01 -1.00000000e-01 -1.00000000e-01
  -1.00000000e-01 -1.

## 6. For a Model $\alpha$, where $\alpha$ = I-IV  and $C = C_{\alpha}^{*}$, estimate on the test set: (a) percent of incorrectly classified examples (b) precision and recall (c) $F_{\beta}$ Make a decision, which one of all models is the best one. Explain your choice.

In [ ]:
# Best C values obtained in previous steps
best_c_linear = best_c
best_c_poly = models_by_kernel['poly'][0].C
best_c_rbf = models_by_kernel['rbf'][0].C
best_c_sigmoid = models_by_kernel['sigmoid'][0].C

# Train the best models with the selected C values
best_models = [
    ('linear', SVC(kernel='linear', C=best_c_linear, random_state=1).fit(X_train_std, y_train)),
    ('poly', SVC(kernel='poly', C=best_c_poly, random_state=1).fit(X_train_std, y_train)),
    ('rbf', SVC(kernel='rbf', C=best_c_rbf, random_state=1).fit(X_train_std, y_train)),
    ('sigmoid', SVC(kernel='sigmoid', C=best_c_sigmoid, random_state=1).fit(X_train_std, y_train))
]

beta = 0.5 #chosen beta

# Evaluate the models on the test set
for kernel, model in best_models:
    y_pred_test = model.predict(X_test_std)

    # Calculate the error rate, precision, recall, and F_beta measure
    error_rate_test = 1 - accuracy_score(y_test, y_pred_test)
    precision_test = precision_score(y_test, y_pred_test, pos_label='ncp')
    recall_test = recall_score(y_test, y_pred_test, pos_label='ncp')
    f_beta_test = (1 + beta**2) * (precision_test * recall_test) / ((beta**2 * precision_test) + recall_test)

    # Print the results
    print(f"Kernel: {kernel}")
    print(f"  (a) Error rate on test set: {error_rate_test:.2f}")
    print(f"  (b) Precision: {precision_test:.2f}, Recall: {recall_test:.2f}")
    print(f"  (c) F_{beta} measure: {f_beta_test:.2f}\n")

Kernel: linear
  (a) Error rate on test set: 0.02
  (b) Precision: 0.98, Recall: 0.98
  (c) F_0.5 measure: 0.98

Kernel: poly
  (a) Error rate on test set: 0.12
  (b) Precision: 0.83, Recall: 0.98
  (c) F_0.5 measure: 0.86

Kernel: rbf
  (a) Error rate on test set: 0.02
  (b) Precision: 0.98, Recall: 0.98
  (c) F_0.5 measure: 0.98

Kernel: sigmoid
  (a) Error rate on test set: 0.02
  (b) Precision: 0.98, Recall: 0.98
  (c) F_0.5 measure: 0.98



Based on the performance metrics (error rate, precision, recall, and $F_\beta$ measure) on the test set, we decide which model is the best one.

The best model has the lowest error rate and highest $F_\beta$ measure. In my specific problem, high precision value is also prioritized.

Due to error rate and $F_\beta$ score, we eliminate the polynomial kernel.

Linear kernel is preferrable due to interpretability due to ability to directly obtain the weight vector, while RBF kernel is better in capturing complex non-linear relationships between the input features and the output variable. Sigmoid kernel is also good at capturing non-linear relationships, but it is too sensitive to hyperparameter choice and computationally more expensive than the other two. So, we eliminate the sigmoid kernel as well.

Since both linear and rbf kernels give the same performance, I would choose linear due to its simplicity and interpretability as interpretability of results is very important in studies involving the medicine and biological sciences.